<a href="https://colab.research.google.com/github/MuhammadAbuSufian/colab/blob/main/PickScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
import requests
from io import BytesIO

model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

[0.5843693614006042, 0.41563060879707336]


In [ ]:
# Load model
device = "cuda"
processor_name_or_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
model_pretrained_name_or_path = "yuvalkirstain/PickScore_v1"

processor = AutoProcessor.from_pretrained(processor_name_or_path)
model = AutoModel.from_pretrained(model_pretrained_name_or_path).eval().to(device)

In [ ]:
# Define the function to calculate probs

def calc_probs(prompt, images):
    # Preprocess
    image_inputs = processor(
        images=images,
        padding=True,
        truncation=True,
        max_length=77,
        return_tensors="pt",
    ).to(device)

    text_inputs = processor(
        text=prompt,
        padding=True,
        truncation=True,
        max_length=77,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        # Embed
        image_embs = model.get_image_features(**image_inputs)
        image_embs = image_embs / torch.norm(image_embs, dim=-1, keepdim=True)

        text_embs = model.get_text_features(**text_inputs)
        text_embs = text_embs / torch.norm(text_embs, dim=-1, keepdim=True)

        # Score
        scores = model.logit_scale.exp() * (text_embs @ image_embs.T)[0]

        # Get probabilities if you have multiple images to choose from
        probs = torch.softmax(scores, dim=-1)

    return probs.cpu().tolist()

In [ ]:
# Example image URLs
image_urls = [
    'http://images.cocodataset.org/val2014/COCO_val2014_000000555472.jpg',
    'http://images.cocodataset.org/val2014/COCO_val2014_000000311295.jpg',
]

# Load images from URLs
images = []
for url in image_urls:
    response = requests.get(url)
    if response.status_code == 200:
        # Convert the response content to a PIL image
        image = Image.open(BytesIO(response.content))
        images.append(image)
    else:
        print(f"Failed to load image from URL: {url}")


[0.8032723665237427, 0.19672763347625732]


In [ ]:
# Call calc_probs function with the prompt and images
prompt = "A elephant is waking"
print(calc_probs(prompt, images))